In [ ]:
# Chest X-Ray Classification using an Inception-like Neural Network

## Step 1: Importing necessary libraries and setting up paths

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pathlib

from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential

# Setting image size and batch size
batch_size = 32
img_height = 180
img_width = 180

# Directory to the dataset (modify this path according to your setup)
data_dir = "chest_xray_dataset/train"

# Loading the dataset with a 20% validation split
train_ds = tf.keras.utils.image_dataset_from_directory(
	data_dir,
	validation_split=0.2,
	subset="training",
	seed=123,
	image_size=(img_height, img_width),
	batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
	data_dir,
	validation_split=0.2,
	subset="validation",
	seed=123,
	image_size=(img_height, img_width),
	batch_size=batch_size)

# Retrieve class names
class_names = train_ds.class_names
print(f"Class names: {class_names}")

In [ ]:
## Step 2: Visualizing the dataset

plt.figure(figsize=(20, 20))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
        
# Save visualizations of some sample images
plt.savefig("directory to save visualization!")
plt.show()

In [ ]:
## Step 3: Defining the Inception-like model

# Normalizing the dataset for better performance
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Define an inception module
def inception_module(input_tensor, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
    conv_1x1 = tf.keras.layers.Conv2D(filters_1x1, (1,1), padding="same", activation="relu")(input_tensor)
    
    conv_3x3 = tf.keras.layers.Conv2D(filters_3x3_reduce, (1,1), padding="same", activation='relu')(input_tensor)
    conv_3x3 = tf.keras.layers.Conv2D(filters_3x3, (3,3), padding="same", activation="relu")(conv_3x3)

    conv_5x5 = tf.keras.layers.Conv2D(filters_5x5_reduce, (1,1), padding="same", activation='relu')(input_tensor)
    conv_5x5 = tf.keras.layers.Conv2D(filters_5x5, (5,5), padding="same", activation="relu")(conv_5x5)

    pool_proj = tf.keras.layers.MaxPooling2D((3,3), strides=(1,1), padding="same")(input_tensor)
    pool_proj = tf.keras.layers.Conv2D(filters_pool_proj, (1,1), padding="same", activation="relu")(pool_proj)

    output = tf.keras.layers.concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj])
    
    return output

# Build the Inception-like model
def build_inception_like_model(input_shape=(img_height, img_width, 3)):
    input_tensor = tf.keras.layers.Input(shape=input_shape)

    x = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2), padding="same", activation="relu")(input_tensor)
    x = tf.keras.layers.MaxPooling2D((3,3),  strides=(2,2), padding="same")(x)

    x = tf.keras.layers.Conv2D(64, (1,1), padding="same", activation="relu")(x)
    x = tf.keras.layers.Conv2D(192, (3,3), padding="same", activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding="same")(x)

    x = inception_module(x, filters_1x1=64, filters_3x3_reduce=96, filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool_proj=32)
    x = inception_module(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool_proj=64)

    x = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding="same")(x)

    x = inception_module(x, filters_1x1=192, filters_3x3_reduce=96, filters_3x3=208, filters_5x5_reduce=16, filters_5x5=48, filters_pool_proj=64)
    x = inception_module(x, filters_1x1=160, filters_3x3_reduce=112, filters_3x3=224, filters_5x5_reduce=24, filters_5x5=64, filters_pool_proj=64)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    output_tensor = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=input_tensor, outputs=output_tensor)
    
    return model

# Create the Inception-like model
inception_like_model = build_inception_like_model(input_shape=(img_height, img_width, 3))

In [ ]:
## Step 4: Compiling and summarizing the model

# Compile the model
inception_like_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss="binary_crossentropy", metrics=['accuracy'])

# Save the model summary to a text file
def save_model_summary(model, filename):
    with open(filename, 'w') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))

save_model_summary(inception_like_model, 'chest_xray_model_summary.txt') # change the directory to save the model to your local folder

# Display the model summary
inception_like_model.summary()

In [ ]:
## Step 5: Training the model

# EarlyStopping callback to prevent overfitting
early_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=1,
    mode="auto",
    restore_best_weights=True)

# Train the model for 100 epochs (stopping early if overfitting starts)
epochs = 100
model_progress = inception_like_model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[early_callback])

In [ ]:
## Step 6: Saving the trained model

# Save the trained model
inception_like_model.save("chest_xray_prediction_model.keras") # you can modify the directory to save the model in your local folder

In [ ]:
## Step 7: Visualizing training results

# Retrieve accuracy and loss data from training history
acc = model_progress.history['accuracy']
val_accuracy = model_progress.history['val_accuracy']

loss = model_progress.history['loss']
val_loss = model_progress.history['val_loss']

# Number of epochs the model was trained for
epochs_trained = len(model_progress.history['loss'])
epochs_range = range(epochs_trained)

# Plot training and validation accuracy/loss
plt.figure(figsize=(10, 10))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_accuracy, label="Validation Accuracy")
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc='upper right')
plt.title("Training and Validation Loss")

# Save the training graph
plt.savefig("directory to save the picture!")
plt.show()